In [ ]:
import os
from dotenv import load_dotenv
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

load_dotenv()

# Configure OpenAI and Vector settings

In [ ]:
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_search_endpoint = os.getenv('AZURE_AI_SEARCH_URL')
azure_search_key = os.getenv('AZURE_AI_SEARCH_API_KEY')
model_deployment_1 : str = "gpt-4"
model_deployment_2: str = "text-embedding-ada-002"
version : str = "2023-05-15"
index_name: str = "langchain-vector"

# Initialize gpt-4, embedding model and connect to Azure AI Search

In [ ]:
model = AzureChatOpenAI(
    openai_api_version=version,
    azure_deployment=model_deployment_1,
)

message = HumanMessage(
    content="Translate this sentence from English to Dutch. I love programming."
)
print(model([message]))

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=model_deployment_2,
    openai_api_version=version,
)

text = "This is a test document."
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])
print(query_result)

vector_store = AzureSearch(azure_search_endpoint=azure_search_endpoint,
                 azure_search_key=azure_search_key,
                 index_name=index_name,
                 embedding_function=embeddings.embed_query)

# Insert text and embeddings into vector store

In [ ]:
loader = TextLoader("data/speech/state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

# Perform a vector similarity search

In [ ]:
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

# Perform a vector similarity search with relevance scores

In [ ]:
docs_and_scores = vector_store.similarity_search_with_relevance_scores(
    query="What did the president say about Ketanji Brown Jackson",
    k=4,
    score_threshold=0.80,
)
from pprint import pprint

pprint(docs_and_scores)

# Perform a Hybrid Search

In [ ]:
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="hybrid",
)
print(docs[0].page_content)